In [17]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.circuit.library import MultiplierGate# IntegerMultiplierをインポート

In [63]:

def create_integer_multiplier_circuit(value_A, value_B):
    """
    2つの古典的な整数 value_A と value_B の積を計算する量子回路を構築します。

    Args:
        value_A (int): 乗算の最初の整数 (A).
        value_B (int): 乗算の2番目の整数 (B).

    Returns:
        QuantumCircuit: 構築された量子乗算回路.
    """
    # 各入力値を表現するために必要な量子ビット数を決定します。
    # .bit_length() は整数を表すのに必要な最小ビット数を返します。
    num_qubits_state = max(value_A.bit_length(), value_B.bit_length()) if value_A + value_B > 0 else 1

    # 積の最大ビット数を決定します。
    # nビットとmビットの数の積は、最大でn+mビットになる可能性があります。
    product = value_A * value_B
    num_qubits_C = product.bit_length() if product else 1

    # IntegerMultiplierの補助ビットは、通常出力レジスタと同じサイズです。
    num_qubits_aux = num_qubits_C

    # 量子レジスタの定義
    # q_A: 最初の入力値 A をエンコードするレジスタ
    q_A = QuantumRegister(num_qubits_state, 'A')
    # q_B: 2番目の入力値 B をエンコードするレジスタ
    q_B = QuantumRegister(num_qubits_state, 'B')
    # q_C: 乗算結果 A * B を格納するレジスタ
    q_C = QuantumRegister(num_qubits_C, 'C')

    # 古典レジスタ: 結果を測定するためのもの
    c_result = ClassicalRegister(num_qubits_C, 'c_result')

    # 量子回路を作成
    qc = QuantumCircuit(q_A, q_B, q_C, c_result)
    qc.name = f"Integer_Multiplier_Circuit_for_{value_A}x{value_B}"

    # --- 1. 古典的な入力値を量子レジスタにエンコード ---
    # 量子ビットは通常 |0...0> で初期化されるため、
    # '1' ビットに対応する位置に X ゲートを適用して値を設定します。
    # Qiskitの算術ゲートは通常リトルエンディアン (最下位ビットがインデックス0) を使用します。
    binary_A = format(value_A, f'0{num_qubits_state}b')[::-1] # ビット列を反転してリトルエンディアンに
    binary_B = format(value_B, f'0{num_qubits_state}b')[::-1]

    # qc.comment(f"--- 入力 {value_A} を A レジスタにエンコード ---")
    for i in range(num_qubits_state):
        if binary_A[i] == '1':
            qc.x(q_A[i])
    qc.barrier()

    # qc.comment(f"--- 入力 {value_B} を B レジスタにエンコード ---")
    for i in range(num_qubits_state):
        if binary_B[i] == '1':
            qc.x(q_B[i])
    qc.barrier()

    # --- 2. IntegerMultiplier を使用して乗算を実行 ---
    # qc.comment(f"--- IntegerMultiplier による {value_A} * {value_B} の計算 ---")
    
    # IntegerMultiplierをインスタンス化します。
    # 引数は (num_qubits_A, num_qubits_B, num_qubits_C) です。
    multiplier = MultiplierGate(num_qubits_state, num_qubits_C)
    
    # multiplier ゲートを回路に追加します。
    # .append() メソッドは、ゲートとその引数となる量子ビットをリストで受け取ります。
    # ここでの量子ビットの順序は、IntegerMultiplierが期待する順序 (q_A, q_B, q_C, q_aux) に従います。
    qc.append(multiplier, q_A[:] + q_B[:] + q_C[:])
    qc.barrier()

    # --- 3. 結果の測定 ---
    # qc.comment("--- 結果を古典レジスタに測定 ---")
    qc.measure(q_C, c_result)

    return qc


In [64]:
# --- 使用例 ---
VALUE_A = 3
VALUE_B = 5

# 古典的な乗算結果
classical_product = VALUE_A * VALUE_B
print(f"古典的な {VALUE_A} * {VALUE_B} の結果: {classical_product}")

# 量子回路の構築
multiplier_circuit = create_integer_multiplier_circuit(VALUE_A, VALUE_B)

# --- 回路のシミュレーション ---
simulator = AerSimulator()
compiled_circuit = transpile(multiplier_circuit, simulator)

# 回路図を表示したい場合 (matplotlib が必要)
# print(compiled_circuit.draw(output='text', idle_wires=False))
# multiplier_circuit.draw(output='mpl', filename='integer_multiplier_circuit.png', fold=-1, idle_wires=False)


# シミュレーションを実行
shots = 1024 # 測定回数
job = simulator.run(compiled_circuit, shots=shots)
result = job.result()
counts = result.get_counts(compiled_circuit)

# 結果の表示と検証
print("\n量子シミュレーション結果 (最も多く観測された状態):")
# 最も多く観測されたビット列を取得
most_common_bitstring = max(counts, key=counts.get)

# Qiskitの測定結果は通常ビッグエンディアン (最上位ビットが文字列の最初) で表示されますが、
# 算術ゲートはリトルエンディアン (最下位ビットがインデックス0) で動作します。
# したがって、ビット列を10進数に変換する際には、ビット列を反転させる必要があります。
# 例: '01111' (ビッグエンディアン) -> '11110' (リトルエンディアン)
# これをint(..., 2)で変換することで、正しい数値を得ます。
simulated_product = int(most_common_bitstring[::-1], 2)

print(f"ビット列: {most_common_bitstring} (ビッグエンディアン)")
print(f"観測回数: {counts[most_common_bitstring]}/{shots}")
print(f"10進数に変換: {simulated_product}")

# 古典的な結果との比較
if simulated_product == classical_product:
    print("\n量子シミュレーション結果は古典的な結果と一致します。期待通りに動作しています。")
else:
    print("\n量子シミュレーション結果は古典的な結果と一致しません。問題がある可能性があります。")

# 別の例
print("\n--- 別の例: 7 * 7 ---")
VALUE_A_2 = 7
VALUE_B_2 = 7
classical_product_2 = VALUE_A_2 * VALUE_B_2
print(f"古典的な {VALUE_A_2} * {VALUE_B_2} の結果: {classical_product_2}")
multiplier_circuit_2 = create_integer_multiplier_circuit(VALUE_A_2, VALUE_B_2)
compiled_circuit_2 = transpile(multiplier_circuit_2, simulator)
job_2 = simulator.run(compiled_circuit_2, shots=shots)
result_2 = job_2.result()
counts_2 = result_2.get_counts(compiled_circuit_2)
most_common_bitstring_2 = max(counts_2, key=counts_2.get)
simulated_product_2 = int(most_common_bitstring_2[::-1], 2)
print(f"量子シミュレーション結果 (10進数): {simulated_product_2}")
if simulated_product_2 == classical_product_2:
    print("量子シミュレーション結果は古典的な結果と一致します。期待通りに動作しています。")
else:
    print("量子シミュレーション結果は古典的な結果と一致しません。問題がある可能性があります。")

古典的な 3 * 5 の結果: 15

量子シミュレーション結果 (最も多く観測された状態):
ビット列: 1111 (ビッグエンディアン)
観測回数: 1024/1024
10進数に変換: 15

量子シミュレーション結果は古典的な結果と一致します。期待通りに動作しています。

--- 別の例: 7 * 7 ---
古典的な 7 * 7 の結果: 49
量子シミュレーション結果 (10進数): 35
量子シミュレーション結果は古典的な結果と一致しません。問題がある可能性があります。
